# Cleaning & PII Redaction

Detect and redact PII fields (SSNs, addresses) using regex + pretrained NER models.

In [ ]:
def redact_pii(text):
    if text is None:
        return text
    
    text = re.sub(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", "[EMAIL_REDACTED]", text)
    text = re.sub(r"\b\d{3}-\d{2}-\d{4}\b", "[SSN_REDACTED]", text)
    text = re.sub(r"\b\d{10}\b", "[PHONE_REDACTED]", text)
    text = re.sub(r"\b\d{3}-\d{3}-\d{4}\b", "[PHONE_REDACTED]", text)
    return text

redact_udf = udf(redact_pii, StringType())

silver_df = silver_df.withColumn("text_clean", redact_udf(col("text_raw")))
silver_path = "abfss://contracts@ragstorage4122025.dfs.core.windows.net/silver/"

(
    silver_df.write
        .format("delta")
        .mode("overwrite")
        .save(silver_path)
)
